In [7]:
import torch
import torch.nn as nn
import numpy as np

In [21]:
def build_nn():
    input_size = 8 * 9
    output = 8
    layer_sizes = [input_size] + [64, 64] + [output]

    assert len(layer_sizes) > 1
    layers = []
    for index in range(len(layer_sizes) - 1):
        linear = nn.Linear(layer_sizes[index], layer_sizes[index + 1])
        act = nn.ReLU() if index < len(layer_sizes) - 2 else nn.Identity()
        layers += (linear, act)
    return nn.Sequential(*layers)

In [40]:
model = build_nn()

In [42]:
tensor = torch.rand(72)

In [45]:
max(model.forward(tensor))

tensor(0.0840, grad_fn=<UnbindBackward0>)

In [48]:
0.0840 * 0.99 + -50

-49.91684